In [17]:
import pandas as pd

In [18]:
sale_data = pd.read_csv('./sale_data.csv', dtype=str)

In [19]:
sale_future_data = pd.read_csv('./sale_future_data.csv', dtype=str)

In [4]:
sale_data.columns

Index(['鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區', '非都市土地使用分區',
       '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態', '主要用途', '主要建材',
       '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛',
       '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別', '車位移轉總面積(平方公尺)',
       '車位總價元', '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯', '移轉編號'],
      dtype='object')

In [20]:
def encode_city_land_usage(x):
    # Encode {'其他': 0, '住': 1, '農': 2, '工': 3, '商': 4, 
    #         '住商': 5, '其他住宅': 6, nan: nan}

    # nan remains
    if not isinstance(x, str):
        return x

    if '非都市' in x:
        return ""

    if '其他' in x:
        end_idx = x.find('(') if '(' in x else len(x) + 1
        if '住商' in x[:end_idx] or '住宅商業' in x[:end_idx]:
            return 5
        if '住宅' in x[:end_idx] or '住' in x[:end_idx]:
            return 6
        return 0

    if '住' in x:
        return 1
    elif '農' in x:
        return 2
    elif '工' in x:
        return 3
    elif '商' in x:
        return 4
    else:
        # There is no such case in the data
        raise ValueError('Unexpected 都市使用分區:', x)

In [41]:
def clean_invalid(dataframe):
    """ Clean invalid data

    *** This is a prerequisite of the following code cells ***
    We transform the features based on the clean data
    -------------------------------------------------------
    Question: 是不是因為清完資料，都市住宅分區的 unique 才會變少？Yes
    """
    dataframe = dataframe[(dataframe['交易標的']!='土地') & (~dataframe['交易標的'].isna())]
    dataframe['Month'] = dataframe['交易年月日'].str[:-2].astype('float')
    dataframe = dataframe.query("Month>=10601 and Month<=11003 ")
    dataframe = dataframe[(dataframe.Month!=10600) & (dataframe.Month!=10700) & \
                          (dataframe.Month!=10800) & (dataframe.Month!=10900) & (dataframe.Month!=11000)]

    return dataframe

In [21]:
data = sale_data['都市土地使用分區'].apply(encode_city_land_usage)

In [22]:
data2 = sale_future_data['都市土地使用分區'].apply(encode_city_land_usage)

In [52]:
sale_data = clean_invalid(sale_data)
none_elevator_df = sale_data.loc[sale_data['電梯'].isna()]
# none_elevator_df.shape[0]

print('Q: 有多少交易標的中，電梯的欄位是 nan?')
for val in none_elevator_df['交易標的'].unique():
    n_samples = none_elevator_df.loc[none_elevator_df['交易標的'] == val].shape[0]
    print(val, n_samples)

print('-'*20)
print('Q: 這些電梯欄位是 nan 的資料是分佈在哪些年月？')
for i, trading_time in enumerate(none_elevator_df['Month'].unique(), start=1):
    print(i, trading_time)

Q: 有多少交易標的中，電梯的欄位是 nan?
房地(土地+建物) 451245
車位 16429
房地(土地+建物)+車位 296317
建物 5393
--------------------
Q: 這些電梯欄位是 nan 的資料是分佈在哪些年月？
1 10601.0
2 10602.0
3 10603.0
4 10604.0
5 10605.0
6 10606.0
7 10607.0
8 10608.0
9 10609.0
10 10610.0
11 10611.0
12 10612.0
13 10701.0
14 10702.0
15 10707.0
16 10711.0
17 10703.0
18 10704.0
19 10705.0
20 10706.0
21 10708.0
22 10805.0
23 10709.0
24 10710.0
25 10712.0
26 10801.0
27 10804.0
28 10802.0
29 10803.0
30 10806.0
31 10807.0
32 10808.0
33 10811.0
34 10809.0
35 10810.0
36 10812.0
37 10901.0
38 10902.0
39 10905.0
40 10903.0
41 10904.0
42 10906.0
43 10907.0
44 10908.0
45 10911.0
46 10910.0
47 10909.0
48 10912.0
49 11001.0
50 11002.0
51 11003.0


In [11]:
print(data)

0            4
1            1
2            1
3            1
4            1
          ... 
1840480    NaN
1840481    NaN
1840482    NaN
1840483    NaN
1840484    NaN
Name: 都市土地使用分區, Length: 1840485, dtype: object


In [ ]:
sale_data['鄉鎮市區'].unique()[:30]

In [ ]:
sale_data['土地位置建物門牌'].unique()[:15]

In [ ]:
sale_data.loc[sale_data['非都市土地使用分區'] == '森林區']['建物型態'].unique()

In [ ]:
sale_data['非都市土地使用編定'].unique()

In [ ]:
count = 0
for data in sale_data['都市土地使用分區']:
    if not isinstance(data, str):
        continue

    if '非都市' in data:
        print(data)
        count += 1

print(count)

In [ ]:
sale_data['建物型態'].unique()

In [ ]:
sale_data.loc[sale_data['建物型態'] == '透天厝']['移轉層次'].unique()

In [ ]:
sale_data.loc[sale_data['建物型態'] == '住宅大樓(11層含以上有電梯)']['移轉層次'].unique()

In [ ]:
sale_future_data.loc[sale_future_data['建物型態'] == '透天厝']['移轉層次'].unique()

In [ ]:
sale_data['車位類別'].unique()

In [ ]:
sale_data['交易筆棟數'].unique()[:30]

In [ ]:
count = 0
for data in sale_data.:
    if '車位0' in sale_data['交易筆棟數']:
        print(data)
        count += 1
        
    if count > 100:
        break

In [ ]:
sale_data['車位類別'].unique()

In [ ]:
sale_data['交易標的'].unique()

In [ ]:
sale_data.loc[sale_data['交易標的'] == '土地'].shape

In [ ]:
sale_data['建物現況格局-廳'].unique()

In [ ]:
sale_data['建物現況格局-房'].unique()

In [ ]:
sale_data.loc[sale_data['建物現況格局-房'] == '250']

In [ ]:
sale_data['非都市土地使用編定'].unique()

In [ ]:
sale_data['土地位置建物門牌'].unique()

In [ ]:
sale_data.loc[sale_data['土地位置建物門牌'] == '臺北市中正區和平西路一段６１號地下１.2.3.4層']

In [ ]:
sale_data['交易筆棟數'].unique().shape
print(sale_data['交易筆棟數'].unique()[:50])

In [ ]:
for row in sale_data['備註'].unique()[:1000]:
    print(row)

In [ ]:
sale_data['主要用途'].unique()[:100]